In [ ]:
#XML reader and Pytrec
!pip install xmltodict pytrec_eval

In [ ]:
#packages
import re
import string
from collections import defaultdict
from math import log

import nltk
import xmltodict
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from pytrec_eval import RelevanceEvaluator
from google.colab import drive

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

**Read Cranfield XML file**

In [48]:
def read_cranfield_xml(xml_file):
  with open(xml_file) as f:
      xml = f.read()

  _dict = xmltodict.parse(xml, attr_prefix="")
  return _dict

**Get Cranfield document collection**

In [44]:

def get_documents_collection(doc):
    dids = []
    docs = []

    root_value = doc.get("root")

    if not isinstance(root_value, dict):
        return dids, docs

    doc_value = root_value.get("doc")

    if not isinstance(doc_value, list):
        return dids, docs

    for item in doc_value:
        if isinstance(item, dict) and 'docno' in item and 'title' in item and 'text' in item:
            dids.append(item['docno'])
            docs.append(str(item['title']) + " " + str(item['text']))

    return dids, docs


Get cranfield query collection

In [43]:
def get_query_collection(doc):
    dids = []
    docs = []

    if "xml" in doc and isinstance(doc["xml"], dict):
        top = doc["xml"].get("top", [])
        for item in top:
            if isinstance(item, dict):
                num = item.get("num")
                title = item.get("title")
                if num is not None and title is not None:
                    dids.append(num)
                    docs.append(str(title))

    return dids, docs


**Get Ground truth /Relevance judgements from cranqrel.trec.txt**

In [39]:
from collections import defaultdict

def read_qrels_file(file_path):
    ground_truth = defaultdict(dict)
    with open(file_path, 'r') as file:
        for line in file:
            query_id, _, doc_id, relevance = line.split(None, 3)  # Split at whitespace, maximum 3 splits
            ground_truth[query_id][doc_id] = int(relevance)
    return ground_truth


# **Preprocessing text**

In [40]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import string

def preprocess_text(docs):
    # Step 1: Join text
    text = ' '.join(map(str, docs))

    # Step 2: Normalize to lowercase
    text = text.lower()

    # Step 3: Tokenize
    tokens = word_tokenize(text)

    # Step 4: Remove punctuation and stop words, and apply stemming and lemmatization
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    preprocessed_tokens = []
    for token in tokens:
        # Remove punctuation
        token = token.translate(str.maketrans('', '', string.punctuation))

        # Remove non-alphabetic tokens and stop words
        if token.isalpha() and token not in stop_words:
            # Stem and lemmatize
            token = lemmatizer.lemmatize(stemmer.stem(token))
            preprocessed_tokens.append(token)

    return preprocessed_tokens


**Calculate Term freequency**

In [41]:
from collections import Counter

def calc_term_frequency(tokens):
    tf_score = Counter(tokens)
    return tf_score

**Calculate Inverted Index**

In [46]:
def build_inverted_index(dids, documents):
    inverted_index = defaultdict(dict)
    doc_lengths = {}

    for doc_id, doc_text in zip(dids, documents):
        tokens = preprocess_text(doc_text)
        doc_lengths[doc_id] = len(tokens)

        # Calculate term frequency and update inverted index
        for term, freq in calc_term_frequency(tokens).items():
            inverted_index[term][doc_id] = freq

    return inverted_index, doc_lengths


**Function to calculate inverse document frequency (IDF)**

In [ ]:
def calculate_tf_idf(tf, df, num_docs):
    return (1 + log(tf)) * log(num_docs / df)

**Function to get dictionary of tf_id**

In [ ]:
def build_tf_idf_index(inverted_index, doc_lengths, num_docs):
    tf_idf_index = defaultdict(dict)

    for term, postings in inverted_index.items():
        df = len(postings)
        for doc_id, tf in postings.items():
            tf_idf_index[term][doc_id] = calculate_tf_idf(tf, df, num_docs) / doc_lengths[doc_id]

    return tf_idf_index

**Calculate using Vector Space Model**

In [47]:
def query_processing_VSM(query, inverted_index, tf_idf_index, doc_lengths, num_docs):
    query_terms = preprocess_text(query)
    query_vector = Counter(query_terms)

    scores = defaultdict(float)

    for term, query_tf in query_vector.items():
        if term in inverted_index:
            for doc_id, doc_tf_idf in tf_idf_index[term].items():
                scores[doc_id] += query_tf * doc_tf_idf

    # Normalize scores by document length
    for doc_id, score in scores.items():
        scores[doc_id] /= doc_lengths[doc_id]

    # Rank documents based on scores
    ranked_docs = {doc_id: score for doc_id, score in sorted(scores.items(), key=lambda x: x[1], reverse=True)}

    return ranked_docs


**Calculate Best Matching 25 score**

In [61]:
def calculate_bm25(tf, df, doc_length, avg_doc_length, num_docs, k1, b, idf):
    # BM25 parameters
    k1_times_tf = k1 * tf
    k1_times_1_minus_b = k1 * (1 - b)

    # Calculate BM25 score
    bm25 = idf * (k1_times_tf + 1) / (tf + k1_times_1_minus_b + b * (doc_length / avg_doc_length))

    return bm25

**Build Best Matching 25 model Index**

In [60]:
def build_bm25_index(inverted_index, doc_lengths, num_docs, k1=1.5, b=0.75):
    bm25_index = defaultdict(dict)
    avg_doc_length = sum(doc_lengths.values()) / num_docs

    # Precompute IDF for each term
    idf_dict = {}
    for term, postings in inverted_index.items():
        df = len(postings)
        idf_dict[term] = log((num_docs - df + 0.5) / (df + 0.5) + 1.0)

    # Build BM25 index
    for term, postings in inverted_index.items():
        idf = idf_dict[term]
        for doc_id, tf in postings.items():
            doc_length = doc_lengths[doc_id]
            bm25_index[term][doc_id] = calculate_bm25(tf, len(postings), doc_length, avg_doc_length, num_docs, k1, b, idf)

    return bm25_index


**Calculate BM25 ranking**

In [54]:
def query_processing_bm25(query, inverted_index, doc_lengths, num_docs):
    query_terms = preprocess_text(query)
    scores = defaultdict(float)
    bm25_index = build_bm25_index(inverted_index, doc_lengths, num_docs)  # Precompute BM25 index

    for term in query_terms:
        if term in inverted_index:
            for doc_id, doc_bm25 in bm25_index[term].items():
                scores[doc_id] += doc_bm25

    # Rank documents based on scores
    ranked_docs = {doc_id: score for doc_id, score in sorted(scores.items(), key=lambda x: x[1], reverse=True)}

    return ranked_docs

 Language Model with Dirichlet Smoothing as another language model for ranking the documents. The Dirichlet Smoothing technique is commonly used to address the issue of unseen terms in the query-document retrieval process

In [59]:
def calculate_language_model_prob(term, doc_id, inverted_index, doc_lengths, mu=1000):
    # Precompute values outside the loop
    total_terms = sum(inverted_index[term].values())
    background_prob = total_terms / sum(doc_lengths.values())
    doc_length = doc_lengths[doc_id]

    # Calculate the probability of the term in the language model using Dirichlet Smoothing
    term_freq = inverted_index[term].get(doc_id, 0)

    return (term_freq + mu * background_prob) / (doc_length + mu)


In [58]:
def query_processing_language_model(query, inverted_index, doc_lengths):
    query_terms = preprocess_text(query)
    scores = defaultdict(float)

    for doc_id in doc_lengths:
        score = 1.0
        for term in query_terms:
            prob = calculate_language_model_prob(term, doc_id, inverted_index, doc_lengths)
            score *= prob
        scores[doc_id] = score

    # Rank documents based on scores
    ranked_docs = defaultdict(float, {doc_id: score for doc_id, score in sorted(scores.items(), key=lambda x: x[1], reverse=True)})

    return ranked_docs

**Calculate Indexing, Ranking and evaluation.**



In [42]:
cranfield_collection_path = '/content/cran.all.1400.xml'
documents = read_cranfield_xml(cranfield_collection_path)
dids, docs  = get_documents_collection(documents)

In [62]:
#Read Document cranfield file

cranfield_collection_path = '/content/cran.all.1400.xml'
documents = read_cranfield_xml(cranfield_collection_path)
dids, docs  = get_documents_collection(documents)

# Build the inverted index and calculate document lengths
inverted_index, doc_lengths = build_inverted_index(dids, docs)

# Calculate the number of documents
num_docs = len(documents)

#Read query file
cranfield_collection_path = '/content/cran.qry.xml'
query = read_cranfield_xml(cranfield_collection_path)
qids, querys = get_query_collection(query)

# Build the TF-IDF VSM index
tf_idf_index = build_tf_idf_index(inverted_index, doc_lengths, num_docs)

qrels_file_path = '/content/cranqrel.trec.txt'  # Replace with the actual path to the qrels.text file
ground_truth = read_qrels_file(qrels_file_path)

# Update the data type of the collections to defaultdict(dict)
ranked_docs_vsm_collection = defaultdict(dict)
ranked_docs_bm25_collection = defaultdict(dict)
ranked_docs_language_model_collection = defaultdict(dict)

for qid, query in zip(qids, querys):
    # Ranking using Vector Space Model
    ranked_docs_vsm_collection[qid] = query_processing_VSM(query, inverted_index, tf_idf_index, doc_lengths, num_docs)

    # Process the query and get ranked documents using BM25
    ranked_docs_bm25_collection[qid] = query_processing_bm25(query, inverted_index, doc_lengths, num_docs)

    # Process the query and get ranked documents using the Language Model with Dirichlet Smoothing
    ranked_docs_language_model_collection[qid] = query_processing_language_model(query, inverted_index, doc_lengths)

# Convert the ranked documents to the required format for evaluation
ranked_docs_vsm_trec = {str(qid): {str(doc_id): rank for rank, (doc_id, _) in enumerate(ranked_docs_vsm_collection[qid].items(), 1)} for qid in ranked_docs_vsm_collection}
ranked_docs_bm25_trec = {str(qid): {str(doc_id): rank for rank, (doc_id, _) in enumerate(ranked_docs_bm25_collection[qid].items(), 1)} for qid in ranked_docs_bm25_collection}
ranked_docs_language_model_trec = {str(qid): {str(doc_id): rank for rank, (doc_id, _) in enumerate(ranked_docs_language_model_collection[qid].items(), 1)} for qid in ranked_docs_language_model_collection}

# Prepare the ground truth data for evaluation
qrels = {}
for qid, tuples in ground_truth.items():
    qrels[qid] = {}
    for doc_id, relevance in tuples.items():
        qrels[qid][doc_id] = relevance

# Create the evaluator instance
evaluator = RelevanceEvaluator(qrels, {'map', 'P_5', 'ndcg'})

# Evaluate the ranking models
trec_eval_results_vsm = evaluator.evaluate(ranked_docs_vsm_trec)
trec_eval_results_bm25 = evaluator.evaluate(ranked_docs_bm25_trec)
trec_eval_results_language_model = evaluator.evaluate(ranked_docs_language_model_trec)

# Print the evaluation results
print("TREC Evaluation Results:")
print("+---------+---------+---------+---------+-----------------------+")
print("| Model   | QueryID | DocID   | Rank    | Similarity (Score)    |")
print("+---------+---------+---------+---------+-----------------------+")

# Write the evaluation results to a file
with open("evaluation_results.txt", "w") as f:
    f.write("TREC Evaluation Results:\n")
    f.write("+---------+---------+---------+---------+-----------------------+\n")
    f.write("| Model   | QueryID | DocID   | Rank    | Similarity (Score)    |\n")
    f.write("+---------+---------+---------+---------+-----------------------+\n")

    for query_id in trec_eval_results_vsm.keys():
        for doc_id, rank in ranked_docs_vsm_trec[query_id].items():
            similarity = trec_eval_results_vsm[query_id]['P_5']
            line = f"| VSM     | Q{query_id}  | {doc_id}  | {rank}  | {similarity:0.4f}          |\n"
            print(line, end="")
            f.write(line)

    for query_id in trec_eval_results_bm25.keys():
        for doc_id, rank in ranked_docs_bm25_trec[query_id].items():
            similarity = trec_eval_results_bm25[query_id]['P_5']
            line = f"| BM25    | Q{query_id}  | {doc_id}  | {rank}  | {similarity:0.4f}          |\n"
            print(line, end="")
            f.write(line)

    for query_id in trec_eval_results_language_model.keys():
        for doc_id, rank in ranked_docs_language_model_trec[query_id].items():
            similarity = trec_eval_results_language_model[query_id]['P_5']
            line = f"| Language| Q{query_id}  | {doc_id}  | {rank}  | {similarity:0.4f}          |\n"
            print(line, end="")
            f.write(line)

    f.write("+---------+---------+---------+---------+-----------------------+\n")

Streaming output truncated to the last 5000 lines.
| VSM     | Q168  | 572  | 20  | 0.0000          |
| VSM     | Q168  | 673  | 21  | 0.0000          |
| VSM     | Q168  | 433  | 22  | 0.0000          |
| VSM     | Q168  | 1047  | 23  | 0.0000          |
| VSM     | Q168  | 1248  | 24  | 0.0000          |
| VSM     | Q168  | 1147  | 25  | 0.0000          |
| VSM     | Q168  | 344  | 26  | 0.0000          |
| VSM     | Q168  | 262  | 27  | 0.0000          |
| VSM     | Q168  | 14  | 28  | 0.0000          |
| VSM     | Q168  | 962  | 29  | 0.0000          |
| VSM     | Q168  | 165  | 30  | 0.0000          |
| VSM     | Q168  | 364  | 31  | 0.0000          |
| VSM     | Q168  | 1239  | 32  | 0.0000          |
| VSM     | Q168  | 522  | 33  | 0.0000          |
| VSM     | Q168  | 662  | 34  | 0.0000          |
| VSM     | Q168  | 499  | 35  | 0.0000          |
| VSM     | Q168  | 189  | 36  | 0.0000          |
| VSM     | Q168  | 193  | 37  | 0.0000          |
| VSM     | Q168  | 976  | 3

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
| Language| Q123  | 1040  | 911  | 0.0000          |
| Language| Q123  | 602  | 912  | 0.0000          |
| Language| Q123  | 614  | 913  | 0.0000          |
| Language| Q123  | 656  | 914  | 0.0000          |
| Language| Q123  | 761  | 915  | 0.0000          |
| Language| Q123  | 951  | 916  | 0.0000          |
| Language| Q123  | 1178  | 917  | 0.0000          |
| Language| Q123  | 91  | 918  | 0.0000          |
| Language| Q123  | 51  | 919  | 0.0000          |
| Language| Q123  | 145  | 920  | 0.0000          |
| Language| Q123  | 1397  | 921  | 0.0000          |
| Language| Q123  | 349  | 922  | 0.0000          |
| Language| Q123  | 956  | 923  | 0.0000          |
| Language| Q123  | 1116  | 924  | 0.0000          |
| Language| Q123  | 985  | 925  | 0.0000          |
| Language| Q123  | 1246  | 926  | 0.0000          |
| Language| Q123  | 286  | 927  | 0.0000          |
| Language| Q123  | 449  | 928  | 0.0000          |
| Language

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
| Language| Q219  | 154  | 601  | 0.0000          |
| Language| Q219  | 771  | 602  | 0.0000          |
| Language| Q219  | 954  | 603  | 0.0000          |
| Language| Q219  | 754  | 604  | 0.0000          |
| Language| Q219  | 832  | 605  | 0.0000          |
| Language| Q219  | 484  | 606  | 0.0000          |
| Language| Q219  | 310  | 607  | 0.0000          |
| Language| Q219  | 901  | 608  | 0.0000          |
| Language| Q219  | 334  | 609  | 0.0000          |
| Language| Q219  | 1005  | 610  | 0.0000          |
| Language| Q219  | 892  | 611  | 0.0000          |
| Language| Q219  | 1220  | 612  | 0.0000          |
| Language| Q219  | 699  | 613  | 0.0000          |
| Language| Q219  | 883  | 614  | 0.0000          |
| Language| Q219  | 815  | 615  | 0.0000          |
| Language| Q219  | 1026  | 616  | 0.0000          |
| Language| Q219  | 444  | 617  | 0.0000          |
| Language| Q219  | 466  | 618  | 0.0000          |
| Language